In [1]:
import pandas as pd
import math
import pickle
import datetime
from nltk.stem import PorterStemmer

# read in data

In [2]:
rows = []
with open('forumTest.data', "r") as fp:
    rows = [line.split(" ",1) for line in fp.readlines()]
test = pd.DataFrame(rows,columns=["subject","content"])
rows = []
with open('forumTraining.data', "r") as fp:
    rows = [line.split(" ",1) for line in fp.readlines()]
training = pd.DataFrame(rows,columns=["subject","content"])

# pickle load

In [ ]:
with open('vocabSTEM.pickle','rb') as file:
    vocab = pickle.load(file)
with open('trgroupSTEM.pickle','rb') as file:
    trgroup = pickle.load(file)
with open('probsSTEM.pickle','rb') as file:
    probs = pickle.load(file)
with open('textSTEM.pickle','rb') as file:
    text = pickle.load(file)
with open('wtprobSTEM.pickle','rb') as file:
    wtprob = pickle.load(file)

# pickle save 

In [8]:
with open('vocabSTEm.pickle','wb') as file:
    pickle.dump(vocab,file)
with open('trgroupSTEM.pickle','wb') as file:
    pickle.dump(trgroup,file)
with open('probsSTEM.pickle','wb') as file:
    pickle.dump(probs,file)
with open('textSTEM.pickle','wb') as file:
    pickle.dump(text,file)
with open('wtprobSTEM.pickle','wb') as file:
    pickle.dump(wtprob,file)

# training

In [3]:
#collect the vocab
vocab = []
ps = PorterStemmer()
for k,row in training.iterrows():
    for word in row.content.split(" "):
        word = ps.stem(word) 
        if word not in vocab:
            vocab.append(word)
print('done')

done


In [4]:
#run the pickle instead
#calculate probability estimates 
trgroup = training.groupby('subject')
docs ={}
probs = {g:0 for g in trgroup.groups.keys()}
text = {g:'' for g in trgroup.groups.keys()} 
wtprob = {g:{} for g in trgroup.groups.keys()} 
for g, cont in trgroup:
    print(g)
    docs[g] = cont.content.tolist()
    probs[g] = len(cont)/len(training)
    for cnt in docs[g]:
        text[g] += cnt.strip()+" "
    n = len(text[g].split(' '))
    for word in vocab:
        word = ps.stem(word)
        nk = text[g].count(word)
        wtprob[g][word] = math.log((nk + 1)/(n+len(vocab)))
    print('done with '+g)
print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')

atheism
done with atheism
autos
done with autos
baseball
done with baseball
christianity
done with christianity
cryptology
done with cryptology
electronics
done with electronics
forsale
done with forsale
graphics
done with graphics
guns
done with guns
hockey
done with hockey
mac
done with mac
medicine
done with medicine
mideastpolitics
done with mideastpolitics
motorcycles
done with motorcycles
mswindows
done with mswindows
pc
done with pc
politics
done with politics
religion
done with religion
space
done with space
xwindows
done with xwindows
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


# Classify

In [5]:
win = 0.0
lose = 0.0
otl = 0.0
for idx,tent in test.iterrows():
    maxnb = -9999999999
    maxg = ""
    maxnb2 = maxnb
    maxg2 = ""
    for g in trgroup.groups.keys():
        tot = math.log(probs[g])
        words = tent.content.split(" ")
        for word in words:
            word = ps.stem(word)
            try:
                tot += wtprob[g][word]     
            except KeyError:
                pass
        if tot > maxnb:
            maxnb2 = maxnb
            maxg2 = maxg
            maxnb = tot
            maxg = g
            
        elif tot > maxnb2:
            maxnb2 = tot
            maxg2 = g
    if maxg == tent.subject:
        win += 1.0
    elif maxg2 == tent.subject:
        otl += 1.0
    else:
        lose += 1.0
print("%f%% 1st correct %f%% 2nd correct "%(win/len(test), otl/len(test)))

0.749734% 1st correct 0.098698% 2nd correct 


# Find important words

In [7]:
topLst = [{'subject':'','word':'','score':0}]*30
topG = {g:[{'word':'','score':0}]*5 for g in trgroup.groups.keys()}
for word in vocab:
    if(len(word)<3):
        continue
    tot = 0
    cnt = 0
    for g in trgroup.groups.keys():
        try:
            tot += math.e**wtprob[g][word]
            cnt += 1
        except KeyError as e:
            print(e)
            pass
    if cnt ==0:
        continue
    avg = tot/cnt
    for g in trgroup.groups.keys():
        try:
            v = (math.e**wtprob[g][word])/avg
        except KeyError:
            print(e)
            continue
        for i in range(len(topG[g])):
            if v > topG[g][i]['score']:
                for j in range(len(topG[g])-2,i,-1):
                    topG[g][j] = topG[g][j-1]
                topG[g][i] = {'word':word,'score':v}
                break
        for i in range(len(topLst)):
            if v > topLst[i]['score']:
                for j in range(len(topLst)-2,i,-1):
                    topLst[j] = topLst[j-1]
                topLst[i] = {'subject':g,'word':word,'score':v}
                break
for i in range(len(topLst)-1,-1,-1):
    print("#%d subject:%s word:%s score:%f"%(i+1,topLst[i]['subject'],topLst[i]['word'],topLst[i]['score']))
orderG = list(trgroup.groups.keys())
orderG.sort()
for g in orderG:
    print(g+"~~~~~~~~~~~~~~~~~~~~~")
    for i in range(len(topG[g])-1,-1,-1):
        print("\t%d word:%s score:%f"%(i+1,topG[g][i]['word'],topG[g][i]['score']))

'international'
'international'
'international'
'international'
'international'
'international'
'international'
'international'
'international'
'international'
'international'
'international'
'international'
'international'
'international'
'international'
'international'
'international'
'international'
'international'
'compromis'
'compromis'
'compromis'
'compromis'
'compromis'
'compromis'
'compromis'
'compromis'
'compromis'
'compromis'
'compromis'
'compromis'
'compromis'
'compromis'
'compromis'
'compromis'
'compromis'
'compromis'
'compromis'
'compromis'
'rais'
'rais'
'rais'
'rais'
'rais'
'rais'
'rais'
'rais'
'rais'
'rais'
'rais'
'rais'
'rais'
'rais'
'rais'
'rais'
'rais'
'rais'
'rais'
'rais'
'amus'
'amus'
'amus'
'amus'
'amus'
'amus'
'amus'
'amus'
'amus'
'amus'
'amus'
'amus'
'amus'
'amus'
'amus'
'amus'
'amus'
'amus'
'amus'
'amus'
'invas'
'invas'
'invas'
'invas'
'invas'
'invas'
'invas'
'invas'
'invas'
'invas'
'invas'
'invas'
'invas'
'invas'
'invas'
'invas'
'invas'
'invas'
'invas'
'invas'


'releas'
'impris'
'impris'
'impris'
'impris'
'impris'
'impris'
'impris'
'impris'
'impris'
'impris'
'impris'
'impris'
'impris'
'impris'
'impris'
'impris'
'impris'
'impris'
'impris'
'impris'
'occasion'
'occasion'
'occasion'
'occasion'
'occasion'
'occasion'
'occasion'
'occasion'
'occasion'
'occasion'
'occasion'
'occasion'
'occasion'
'occasion'
'occasion'
'occasion'
'occasion'
'occasion'
'occasion'
'occasion'
'inde'
'inde'
'inde'
'inde'
'inde'
'inde'
'inde'
'inde'
'inde'
'inde'
'inde'
'inde'
'inde'
'inde'
'inde'
'inde'
'inde'
'inde'
'inde'
'inde'
'intoler'
'intoler'
'intoler'
'intoler'
'intoler'
'intoler'
'intoler'
'intoler'
'intoler'
'intoler'
'intoler'
'intoler'
'intoler'
'intoler'
'intoler'
'intoler'
'intoler'
'intoler'
'intoler'
'intoler'
'degre'
'degre'
'degre'
'degre'
'degre'
'degre'
'degre'
'degre'
'degre'
'degre'
'degre'
'degre'
'degre'
'degre'
'degre'
'degre'
'degre'
'degre'
'degre'
'degre'
'devis'
'devis'
'devis'
'devis'
'devis'
'devis'
'devis'
'devis'
'devis'
'devis'
'devis'
'de

'reprehens'
'reprehens'
'reprehens'
'reprehens'
'reprehens'
'reprehens'
'reprehens'
'reprehens'
'reprehens'
'reprehens'
'reprehens'
'reprehens'
'reprehens'
'reprehens'
'reprehens'
'reprehens'
'reprehens'
'reprehens'
'reprehens'
'reprehens'
'repuls'
'repuls'
'repuls'
'repuls'
'repuls'
'repuls'
'repuls'
'repuls'
'repuls'
'repuls'
'repuls'
'repuls'
'repuls'
'repuls'
'repuls'
'repuls'
'repuls'
'repuls'
'repuls'
'repuls'
'loos'
'loos'
'loos'
'loos'
'loos'
'loos'
'loos'
'loos'
'loos'
'loos'
'loos'
'loos'
'loos'
'loos'
'loos'
'loos'
'loos'
'loos'
'loos'
'loos'
'compris'
'compris'
'compris'
'compris'
'compris'
'compris'
'compris'
'compris'
'compris'
'compris'
'compris'
'compris'
'compris'
'compris'
'compris'
'compris'
'compris'
'compris'
'compris'
'compris'
'intercours'
'intercours'
'intercours'
'intercours'
'intercours'
'intercours'
'intercours'
'intercours'
'intercours'
'intercours'
'intercours'
'intercours'
'intercours'
'intercours'
'intercours'
'intercours'
'intercours'
'intercours'
'inter

'unquestion'
'unquestion'
'unquestion'
'unquestion'
'unquestion'
'unquestion'
'unquestion'
'unquestion'
'unquestion'
'unquestion'
'unquestion'
'unquestion'
'exagger'
'exagger'
'exagger'
'exagger'
'exagger'
'exagger'
'exagger'
'exagger'
'exagger'
'exagger'
'exagger'
'exagger'
'exagger'
'exagger'
'exagger'
'exagger'
'exagger'
'exagger'
'exagger'
'exagger'
'emotion'
'emotion'
'emotion'
'emotion'
'emotion'
'emotion'
'emotion'
'emotion'
'emotion'
'emotion'
'emotion'
'emotion'
'emotion'
'emotion'
'emotion'
'emotion'
'emotion'
'emotion'
'emotion'
'emotion'
'cruis'
'cruis'
'cruis'
'cruis'
'cruis'
'cruis'
'cruis'
'cruis'
'cruis'
'cruis'
'cruis'
'cruis'
'cruis'
'cruis'
'cruis'
'cruis'
'cruis'
'cruis'
'cruis'
'cruis'
'wholesal'
'wholesal'
'wholesal'
'wholesal'
'wholesal'
'wholesal'
'wholesal'
'wholesal'
'wholesal'
'wholesal'
'wholesal'
'wholesal'
'wholesal'
'wholesal'
'wholesal'
'wholesal'
'wholesal'
'wholesal'
'wholesal'
'wholesal'
'depos'
'depos'
'depos'
'depos'
'depos'
'depos'
'depos'
'depos'


'gracious'
'gracious'
'gracious'
'gracious'
'gracious'
'gracious'
'gracious'
'gracious'
'gracious'
'gracious'
'gracious'
'gracious'
'gracious'
'gracious'
'gracious'
'gracious'
'gracious'
'computer'
'computer'
'computer'
'computer'
'computer'
'computer'
'computer'
'computer'
'computer'
'computer'
'computer'
'computer'
'computer'
'computer'
'computer'
'computer'
'computer'
'computer'
'computer'
'computer'
'coachhous'
'coachhous'
'coachhous'
'coachhous'
'coachhous'
'coachhous'
'coachhous'
'coachhous'
'coachhous'
'coachhous'
'coachhous'
'coachhous'
'coachhous'
'coachhous'
'coachhous'
'coachhous'
'coachhous'
'coachhous'
'coachhous'
'coachhous'
'ellips'
'ellips'
'ellips'
'ellips'
'ellips'
'ellips'
'ellips'
'ellips'
'ellips'
'ellips'
'ellips'
'ellips'
'ellips'
'ellips'
'ellips'
'ellips'
'ellips'
'ellips'
'ellips'
'ellips'
'alias'
'alias'
'alias'
'alias'
'alias'
'alias'
'alias'
'alias'
'alias'
'alias'
'alias'
'alias'
'alias'
'alias'
'alias'
'alias'
'alias'
'alias'
'alias'
'alias'
'exce'
'exce'

...

'essens'
'essens'
'essens'
'essens'
'essens'
'essens'
'unadulter'
'unadulter'
'unadulter'
'unadulter'
'unadulter'
'unadulter'
'unadulter'
'unadulter'
'unadulter'
'unadulter'
'unadulter'
'unadulter'
'unadulter'
'unadulter'
'unadulter'
'unadulter'
'unadulter'
'unadulter'
'unadulter'
'unadulter'
'wannabe'
'wannabe'
'wannabe'
'wannabe'
'wannabe'
'wannabe'
'wannabe'
'wannabe'
'wannabe'
'wannabe'
'wannabe'
'wannabe'
'wannabe'
'wannabe'
'wannabe'
'wannabe'
'wannabe'
'wannabe'
'wannabe'
'wannabe'
'baptis'
'baptis'
'baptis'
'baptis'
'baptis'
'baptis'
'baptis'
'baptis'
'baptis'
'baptis'
'baptis'
'baptis'
'baptis'
'baptis'
'baptis'
'baptis'
'baptis'
'baptis'
'baptis'
'baptis'
'appointe'
'appointe'
'appointe'
'appointe'
'appointe'
'appointe'
'appointe'
'appointe'
'appointe'
'appointe'
'appointe'
'appointe'
'appointe'
'appointe'
'appointe'
'appointe'
'appointe'
'appointe'
'appointe'
'appointe'
'saduce'
'saduce'
'saduce'
'saduce'
'saduce'
'saduce'
'saduce'
'saduce'
'saduce'
'saduce'
'saduce'
'saduce

'unrepent'
'unrepent'
'unrepent'
'unrepent'
'unrepent'
'unrepent'
'unrepent'
'philosophis'
'philosophis'
'philosophis'
'philosophis'
'philosophis'
'philosophis'
'philosophis'
'philosophis'
'philosophis'
'philosophis'
'philosophis'
'philosophis'
'philosophis'
'philosophis'
'philosophis'
'philosophis'
'philosophis'
'philosophis'
'philosophis'
'philosophis'
'hypthes'
'hypthes'
'hypthes'
'hypthes'
'hypthes'
'hypthes'
'hypthes'
'hypthes'
'hypthes'
'hypthes'
'hypthes'
'hypthes'
'hypthes'
'hypthes'
'hypthes'
'hypthes'
'hypthes'
'hypthes'
'hypthes'
'hypthes'
'cleas'
'cleas'
'cleas'
'cleas'
'cleas'
'cleas'
'cleas'
'cleas'
'cleas'
'cleas'
'cleas'
'cleas'
'cleas'
'cleas'
'cleas'
'cleas'
'cleas'
'cleas'
'cleas'
'cleas'
'mispres'
'mispres'
'mispres'
'mispres'
'mispres'
'mispres'
'mispres'
'mispres'
'mispres'
'mispres'
'mispres'
'mispres'
'mispres'
'mispres'
'mispres'
'mispres'
'mispres'
'mispres'
'mispres'
'mispres'
'agree'
'agree'
'agree'
'agree'
'agree'
'agree'
'agree'
'agree'
'agree'
'agree'
'ag

'dismember'
'dismember'
'pupos'
'pupos'
'pupos'
'pupos'
'pupos'
'pupos'
'pupos'
'pupos'
'pupos'
'pupos'
'pupos'
'pupos'
'pupos'
'pupos'
'pupos'
'pupos'
'pupos'
'pupos'
'pupos'
'pupos'
'permis'
'permis'
'permis'
'permis'
'permis'
'permis'
'permis'
'permis'
'permis'
'permis'
'permis'
'permis'
'permis'
'permis'
'permis'
'permis'
'permis'
'permis'
'permis'
'permis'
'retalli'
'retalli'
'retalli'
'retalli'
'retalli'
'retalli'
'retalli'
'retalli'
'retalli'
'retalli'
'retalli'
'retalli'
'retalli'
'retalli'
'retalli'
'retalli'
'retalli'
'retalli'
'retalli'
'retalli'
'aazaade'
'aazaade'
'aazaade'
'aazaade'
'aazaade'
'aazaade'
'aazaade'
'aazaade'
'aazaade'
'aazaade'
'aazaade'
'aazaade'
'aazaade'
'aazaade'
'aazaade'
'aazaade'
'aazaade'
'aazaade'
'aazaade'
'aazaade'
'guarate'
'guarate'
'guarate'
'guarate'
'guarate'
'guarate'
'guarate'
'guarate'
'guarate'
'guarate'
'guarate'
'guarate'
'guarate'
'guarate'
'guarate'
'guarate'
'guarate'
'guarate'
'guarate'
'guarate'
'appris'
'appris'
'appris'
'appris'


'proposition'
'proposition'
'proposition'
'proposition'
'proposition'
'proposition'
'proposition'
'proposition'
'proposition'
'proposition'
'proposition'
'proposition'
'proposition'
'discriminate'
'discriminate'
'discriminate'
'discriminate'
'discriminate'
'discriminate'
'discriminate'
'discriminate'
'discriminate'
'discriminate'
'discriminate'
'discriminate'
'discriminate'
'discriminate'
'discriminate'
'discriminate'
'discriminate'
'discriminate'
'discriminate'
'discriminate'
'neoliber'
'neoliber'
'neoliber'
'neoliber'
'neoliber'
'neoliber'
'neoliber'
'neoliber'
'neoliber'
'neoliber'
'neoliber'
'neoliber'
'neoliber'
'neoliber'
'neoliber'
'neoliber'
'neoliber'
'neoliber'
'neoliber'
'neoliber'
'nonprolifer'
'nonprolifer'
'nonprolifer'
'nonprolifer'
'nonprolifer'
'nonprolifer'
'nonprolifer'
'nonprolifer'
'nonprolifer'
'nonprolifer'
'nonprolifer'
'nonprolifer'
'nonprolifer'
'nonprolifer'
'nonprolifer'
'nonprolifer'
'nonprolifer'
'nonprolifer'
'nonprolifer'
'nonprolifer'
'holtsing'
'holtsi

	3 word:stephan score:16.724700
	2 word:cramer score:17.378300
	1 word:stephanopoulo score:18.652825
religion~~~~~~~~~~~~~~~~~~~~~
	5 word:decenso score:13.488927
	4 word:kendig score:14.621258
	3 word:astrian score:14.812704
	2 word:zoro score:15.064689
	1 word:zoroast score:15.228465
space~~~~~~~~~~~~~~~~~~~~~
	5 word:satel score:16.343324
	4 word:shuttl score:16.985678
	3 word:spacecraft score:17.641887
	2 word:lunar score:18.147019
	1 word:orbit score:18.170428
xwindows~~~~~~~~~~~~~~~~~~~~~
	5 word:onam score:16.825601
	4 word:printf score:17.047058
	3 word:colormap score:17.104510
	2 word:xmu score:17.727892
	1 word:widget score:18.553812
